In [ ]:
# Model Comparison: XGBoost vs Rule-Based

def calculate_metrics(returns_col, signal_col, df):
    results = {}
    for ticker in df.index.get_level_values('ticker').unique():
        ticker_data = df.xs(ticker, level='ticker').dropna(subset=[returns_col, 'log_return'])
        if len(ticker_data) == 0:
            continue
        
        strat_ret = ticker_data[returns_col]
        buy_hold = ticker_data['log_return']
        signals = ticker_data[signal_col]

        # Cumulative returns
        strat_cum = np.exp(strat_ret.sum()) - 1
        bh_cum = np.exp(buy_hold.sum()) - 1
        sharpe = (strat_ret.mean() / strat_ret.std()) * np.sqrt(252) if strat_ret.std() > 0 else 0
        
        # Max drawdown
        cum_log_rets = strat_ret.cumsum()
        running_max = cum_log_rets.expanding().max()
        drawdown = cum_log_rets - running_max
        max_dd = np.exp(drawdown.min()) - 1
        
        # Win rate
        wins = (strat_ret > 0).sum()
        trades = (signals != 0).sum()
        win_rate = wins / trades if trades > 0 else 0

        results[ticker] = {
            'return': strat_cum,
            'bh_return': bh_cum,
            'sharpe': sharpe,
            'max_dd': max_dd,
            'win_rate': win_rate,
            'trades': trades
        }

    all_returns = df.dropna(subset=[returns_col])[returns_col]
    overall_sharpe = (all_returns.mean() / all_returns.std()) * np.sqrt(252) if all_returns.std() > 0 else 0

    return results, overall_sharpe

xgb_results, xgb_overall = calculate_metrics('strategy_return', 'signal', xgb_model.test_data)
rb_results, rb_overall = calculate_metrics('strategy_return', 'signal', rb_model.data)

for ticker in sorted(xgb_results.keys()): #Print results in a table
    xgb_metrics = xgb_results[ticker]
    rb_metrics = rb_results[ticker]

    print(f"\n{ticker}:")
    print(f"  {'Metric':<20} {'XGBoost':>12} {'Rule-Based':>12} {'Difference':>12}")
    print(f"  {'-'*20} {'-'*12} {'-'*12} {'-'*12}")
    print(f"  {'Strategy Return':<20} {xgb_metrics['return']*100:>11.2f}% {rb_metrics['return']*100:>11.2f}% {(xgb_metrics['return']-rb_metrics['return'])*100:>11.2f}%")
    print(f"  {'Buy & Hold':<20} {xgb_metrics['bh_return']*100:>11.2f}% {rb_metrics['bh_return']*100:>11.2f}%")
    print(f"  {'Sharpe Ratio':<20} {xgb_metrics['sharpe']:>12.2f} {rb_metrics['sharpe']:>12.2f} {xgb_metrics['sharpe']-rb_metrics['sharpe']:>12.2f}")
    print(f"  {'Max Drawdown':<20} {xgb_metrics['max_dd']*100:>11.2f}% {rb_metrics['max_dd']*100:>11.2f}% {(xgb_metrics['max_dd']-rb_metrics['max_dd'])*100:>11.2f}%")
    print(f"  {'Win Rate':<20} {xgb_metrics['win_rate']*100:>11.1f}% {rb_metrics['win_rate']*100:>11.1f}% {(xgb_metrics['win_rate']-rb_metrics['win_rate'])*100:>11.1f}%")
    print(f"  {'Trades':<20} {xgb_metrics['trades']:>12} {rb_metrics['trades']:>12} {xgb_metrics['trades']-rb_metrics['trades']:>12}")

print(f"\n{'='*80}")
print(f"OVERALL SHARPE RATIO:")
print(f"  XGBoost:    {xgb_overall:>6.3f}")
print(f"  Rule-Based: {rb_overall:>6.3f}")



AAPL:
  Metric                    XGBoost   Rule-Based   Difference
  -------------------- ------------ ------------ ------------
  Strategy Return            28.21%       26.50%        1.71%
  Buy & Hold                  6.99%        6.99%
  Sharpe Ratio                 0.46         0.51        -0.05
  Max Drawdown              -27.67%      -24.30%       -3.37%
  Win Rate                    49.4%        51.9%        -2.5%
  Trades                        407          391           16

AMZN:
  Metric                    XGBoost   Rule-Based   Difference
  -------------------- ------------ ------------ ------------
  Strategy Return             7.17%       49.08%      -41.90%
  Buy & Hold                 -8.86%       -8.86%
  Sharpe Ratio                 0.08         0.63        -0.55
  Max Drawdown              -41.59%      -21.52%      -20.07%
  Win Rate                    48.4%        51.6%        -3.2%
  Trades                        467          339          128

GOOGL:
  Metric    

In [ ]:
#Data Visualization
tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SPY']
continued_input = True
while continued_input == True:
    input_ticker = input("Please enter a stock ('AAPL') for example: " )
    for i in range(6):
        if input_ticker == tickers[i]:
            print("Done")
            continued_input = False
    
graphed_df = rb_model.data.xs(input_ticker, level = 0)

strategy_graphed_return_pct = (np.exp(graphed_df['strategy_return'].cumsum()) - 1) * 100

dates = graphed_df.index

xgb_graphed_cum_return_pct = (
    xgb_model.test_data
    .groupby(level='ticker')['strategy_return']
    .apply(lambda x: (np.exp(x.cumsum()) - 1) * 100)
)

xgb_graphed_returns = xgb_graphed_cum_return_pct.xs(input_ticker, level = 0)


class data_analysis:
    line_dict = dict(marker = ".", markersize = 3)

    def __init__(self, x_XGBoost, y_XGBoost, x_RB, y_RB, rb_model):
        self.rbmodel = rb_model
        print("Graphs Based on", input_ticker, "Stock")
        
        self.xgboost_graph(x_XGBoost, y_XGBoost)
        self.rulebased_model_graph(x_RB, y_RB)
        self.combined_graph(x_XGBoost, y_XGBoost, x_RB, y_RB)

    def xgboost_graph(self, x_XGBoost, y_XGBoost):
        plt.title("XGBoost Performance Graph")

        plt.xlabel("Dates")
        plt.ylabel("XGBoost Returns (%)")
        
        plt.plot(dates, xgb_graphed_cum_return_pct[:len(dates)], **self.line_dict)
        plt.tick_params(axis = 'x', which = 'major', labelsize = 8)

        plt.show()

    def rulebased_model_graph(self, x_RB, y_RB):
        plt.title("Rule Based Model Performance Graph")

        plt.xlabel("Dates")
        plt.ylabel("Rule Based Returns (%)")
        
        plt.plot(dates[:len(strategy_graphed_return_pct)], strategy_graphed_return_pct, color='orange', **self.line_dict)
        plt.tick_params(axis = 'x', which = 'major', labelsize = 8)

        plt.show()

    def combined_graph(self, x_XGBoost, y_XGBoost, x_RB, y_RB):
        plt.title("Combined Performance Graph")

        plt.xlabel("Dates")
        plt.ylabel("Strategy Returns (%)")
        
        plt.plot(dates, xgb_graphed_cum_return_pct[:len(dates)], label='XGBoost', **self.line_dict)
        plt.plot(dates[:len(strategy_graphed_return_pct)], strategy_graphed_return_pct, color='orange', label='Rule-Based', **self.line_dict)
        plt.legend()
        
        plt.tick_params(axis = 'x', which = 'major', labelsize = 8)

        plt.show()
        


analyse_data = data_analysis(dates, xgb_graphed_cum_return_pct[:len(dates)], dates[:len(strategy_graphed_return_pct)], strategy_graphed_return_pct, rb_model)